In [1]:
# -*- coding: utf-8 -*-
"""EduTutorAI Optimized with Textbox Outputs"""

!pip install transformers torch gradio -q

import gradio as gr
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# ------------------------------
# Load model and tokenizer
# ------------------------------
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ------------------------------
# Core response generator
# ------------------------------
def generate_response(prompt, max_new_tokens=250):
    start_time = time.time()  # measure speed

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,  # cap output size
            do_sample=False,                # faster, deterministic
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()

    elapsed = round(time.time() - start_time, 2)
    return f"Response Time: {elapsed}s\n\n{response}"

# ------------------------------
# Concept Explanation
# ------------------------------
def concept_explanation(concept):
    prompt = f"Explain the concept of {concept} clearly in simple terms with one or two short examples:"
    return generate_response(prompt, max_new_tokens=200)

# ------------------------------
# Quiz Generator
# ------------------------------
def quiz_generator(concept):
    prompt = f"""Generate exactly 5 quiz questions about {concept}.
Use different types (MCQ, True/False, Short Answer).
At the end, provide an ANSWERS section with correct answers."""
    return generate_response(prompt, max_new_tokens=300)

# ------------------------------
# Gradio Interface
# ------------------------------
with gr.Blocks() as app:
    gr.Markdown("# 📘 EduTutorAI - Optimized Educational Assistant")

    with gr.Tabs():
        with gr.TabItem("Concept Explanation"):
            concept_input = gr.Textbox(label="Enter a concept", placeholder="e.g., machine learning")
            explain_btn = gr.Button("Explain")
            explanation_output = gr.Textbox(label="Explanation", lines=12, interactive=False)

            explain_btn.click(concept_explanation, inputs=concept_input, outputs=explanation_output)

        with gr.TabItem("Quiz Generator"):
            quiz_input = gr.Textbox(label="Enter a topic", placeholder="e.g., physics")
            quiz_btn = gr.Button("Generate Quiz")
            quiz_output = gr.Textbox(label="Quiz Questions", lines=15, interactive=False)

            quiz_btn.click(quiz_generator, inputs=quiz_input, outputs=quiz_output)

app.launch(share=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce7ebf5958fb44dee4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
